In [1]:
import pandas as pd
import re
import preprocessor as p
import string
import emoji
import unicodedata
from nltk.corpus import stopwords
import json

## Importing JSON data

In [2]:
# Reading tweets from JSON file and ading them to Pandas Dataframe
tweets = pd.read_json('saver_output.json')

In [3]:
tweets.head(10)

,id,date,hashtags,text
0,1223757837400186880,2020-02-01 23:59:39+00:00,,Coronavirus i morti salgono a 304 In un giorno...
1,1223757836821368832,2020-02-01 23:59:39+00:00,,Coronavirus sale a 259 il numero dei morti 18 ...
2,1223757835806310400,2020-02-01 23:59:39+00:00,,Coronavirus venerd record con 46 morti e 2102 ...
3,1223757835214913536,2020-02-01 23:59:38+00:00,,Coronavirus tutto quello che sappiamo e ci che...
4,1223757834883674112,2020-02-01 23:59:38+00:00,,Si vabb ci sono pi morti per femminicidio o pe...
5,1223757834602647552,2020-02-01 23:59:38+00:00,,Coronavirus gi 12mila casi nel mondo i dieci P...
6,1223757834065797120,2020-02-01 23:59:38+00:00,,Coronavirus la mascherina serve ma se indossat...
7,1223757833046515712,2020-02-01 23:59:38+00:00,,Coronavirus dopo i due casi accertati 32 perso...
8,1223757821407264768,2020-02-01 23:59:35+00:00,#coronavirus,Se avete un parente un amico o un conoscente c...
9,1223757748585816064,2020-02-01 23:59:18+00:00,#coronarvirusitalia #coronavirus,Quando tutti sono preoccupati per il corona vi...


## Cleaning

### Preprocessor and emoji

In [4]:
# Setting preprocesor options
p.set_options(p.OPT.URL,p.OPT.MENTION)

In [5]:
# Function for cleaning with preprocessor
# It removes URLs and mentions
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

In [6]:
# Demojize
# Replaces emojis with text
def demojize(row):
    text = row['text']
    text = emoji.demojize(text)
    return text

In [7]:
# Applying previously defined functions
tweets['text'] = tweets.apply(preprocess_tweet, axis=1)
tweets['text'] = tweets.apply(demojize, axis=1)

In [8]:
tweets.head(10)

,id,date,hashtags,text
0,1223757837400186880,2020-02-01 23:59:39+00:00,,Coronavirus i morti salgono a 304 In un giorno...
1,1223757836821368832,2020-02-01 23:59:39+00:00,,Coronavirus sale a 259 il numero dei morti 18 ...
2,1223757835806310400,2020-02-01 23:59:39+00:00,,Coronavirus venerd record con 46 morti e 2102 ...
3,1223757835214913536,2020-02-01 23:59:38+00:00,,Coronavirus tutto quello che sappiamo e ci che...
4,1223757834883674112,2020-02-01 23:59:38+00:00,,Si vabb ci sono pi morti per femminicidio o pe...
5,1223757834602647552,2020-02-01 23:59:38+00:00,,Coronavirus gi 12mila casi nel mondo i dieci P...
6,1223757834065797120,2020-02-01 23:59:38+00:00,,Coronavirus la mascherina serve ma se indossat...
7,1223757833046515712,2020-02-01 23:59:38+00:00,,Coronavirus dopo i due casi accertati 32 perso...
8,1223757821407264768,2020-02-01 23:59:35+00:00,#coronavirus,Se avete un parente un amico o un conoscente c...
9,1223757748585816064,2020-02-01 23:59:18+00:00,#coronarvirusitalia #coronavirus,Quando tutti sono preoccupati per il corona vi...


### Re

In [9]:
# Deleting duplicate tweets
tweets = tweets.drop_duplicates(subset='text')
tweets['data'] = tweets['text']

In [10]:
# Defining stopwords and adding a space before and after to exclude the case
# in which the stopword is cointained in a word
words = set(stopwords.words('italian'))
stopwords = [' ' + x + ' ' for x in words]

In [11]:
# Removing the tags
tweets.text = tweets.text.replace(
    "@[\w]*[_-]*[\w]*", " ", regex=True)
# Removing the spaces in excess
tweets.text = tweets.text.replace('\s+', ' ', regex=True)
# Removing the space at the beginning
tweets.text = tweets.text.replace('^ ', '', regex=True)
# Removing the space at the end
tweets.text = tweets.text.replace(' $', '', regex=True)
# To lowercase
tweets.text = tweets.text.apply(
    lambda x: x.lower())
tweets.text = tweets.text.replace('^', ' ', regex=True)
tweets.text = tweets.text.replace('$', ' ', regex=True)

# Removing stopwords
for word in stopwords:
    tweets.text = tweets.text.replace(word, ' ', regex=True)

# Removing the space at the beginning and at the end
tweets.text = tweets.text.apply(lambda x: x.strip())
# Removing empty tweets
tweets = tweets[tweets.text != '']

In [12]:
tweets.head(10)

,id,date,hashtags,text,data
0,1223757837400186880,2020-02-01 23:59:39+00:00,,coronavirus morti salgono 304 giorno 45 decess...,Coronavirus i morti salgono a 304 In un giorno...
1,1223757836821368832,2020-02-01 23:59:39+00:00,,coronavirus sale 259 numero morti 18 casi europa,Coronavirus sale a 259 il numero dei morti 18 ...
2,1223757835806310400,2020-02-01 23:59:39+00:00,,coronavirus venerd record 46 morti 2102 nuovi ...,Coronavirus venerd record con 46 morti e 2102 ...
3,1223757835214913536,2020-02-01 23:59:38+00:00,,coronavirus sappiamo resta ancora capire,Coronavirus tutto quello che sappiamo e ci che...
4,1223757834883674112,2020-02-01 23:59:38+00:00,,vabb pi morti femminicidio incidenti stradali ...,Si vabb ci sono pi morti per femminicidio o pe...
5,1223757834602647552,2020-02-01 23:59:38+00:00,,coronavirus gi 12mila casi mondo dieci paesi p...,Coronavirus gi 12mila casi nel mondo i dieci P...
6,1223757834065797120,2020-02-01 23:59:38+00:00,,coronavirus mascherina serve indossata male,Coronavirus la mascherina serve ma se indossat...
7,1223757833046515712,2020-02-01 23:59:38+00:00,,coronavirus dopo due casi accertati 32 persone...,Coronavirus dopo i due casi accertati 32 perso...
8,1223757821407264768,2020-02-01 23:59:35+00:00,#coronavirus,parente amico conoscente andando pi mangiare r...,Se avete un parente un amico o un conoscente c...
9,1223757748585816064,2020-02-01 23:59:18+00:00,#coronarvirusitalia #coronavirus,quando preoccupati corona virus no perch tanto...,Quando tutti sono preoccupati per il corona vi...


### Further cleaning

In [13]:
# Removing usernames in case preprocessor didn't work
def remove_mentions(text):
    return re.sub(r"(?:\@|https?\://)\S+", "", text)


# Remove punctuation in case Re missed something
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

In [14]:
# Removing mentions
tweets['text'] = tweets['text'].apply(remove_mentions)

# Removing punctuation
tweets['text'] = tweets['text'].apply(remove_punctuations)

In [15]:
tweets.head(10)

,id,date,hashtags,text,data
0,1223757837400186880,2020-02-01 23:59:39+00:00,,coronavirus morti salgono 304 giorno 45 decess...,Coronavirus i morti salgono a 304 In un giorno...
1,1223757836821368832,2020-02-01 23:59:39+00:00,,coronavirus sale 259 numero morti 18 casi europa,Coronavirus sale a 259 il numero dei morti 18 ...
2,1223757835806310400,2020-02-01 23:59:39+00:00,,coronavirus venerd record 46 morti 2102 nuovi ...,Coronavirus venerd record con 46 morti e 2102 ...
3,1223757835214913536,2020-02-01 23:59:38+00:00,,coronavirus sappiamo resta ancora capire,Coronavirus tutto quello che sappiamo e ci che...
4,1223757834883674112,2020-02-01 23:59:38+00:00,,vabb pi morti femminicidio incidenti stradali ...,Si vabb ci sono pi morti per femminicidio o pe...
5,1223757834602647552,2020-02-01 23:59:38+00:00,,coronavirus gi 12mila casi mondo dieci paesi p...,Coronavirus gi 12mila casi nel mondo i dieci P...
6,1223757834065797120,2020-02-01 23:59:38+00:00,,coronavirus mascherina serve indossata male,Coronavirus la mascherina serve ma se indossat...
7,1223757833046515712,2020-02-01 23:59:38+00:00,,coronavirus dopo due casi accertati 32 persone...,Coronavirus dopo i due casi accertati 32 perso...
8,1223757821407264768,2020-02-01 23:59:35+00:00,#coronavirus,parente amico conoscente andando pi mangiare r...,Se avete un parente un amico o un conoscente c...
9,1223757748585816064,2020-02-01 23:59:18+00:00,#coronarvirusitalia #coronavirus,quando preoccupati corona virus no perch tanto...,Quando tutti sono preoccupati per il corona vi...


In [16]:
# Remove accents in order to mitigate weird charachters in the final result
def strip_accents(row):
    text = row['text']
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return text

In [17]:
# Removing accent
tweets['text'] = tweets.apply(strip_accents, axis=1)

In [18]:
tweets.head(10)

,id,date,hashtags,text,data
0,1223757837400186880,2020-02-01 23:59:39+00:00,,coronavirus morti salgono 304 giorno 45 decess...,Coronavirus i morti salgono a 304 In un giorno...
1,1223757836821368832,2020-02-01 23:59:39+00:00,,coronavirus sale 259 numero morti 18 casi europa,Coronavirus sale a 259 il numero dei morti 18 ...
2,1223757835806310400,2020-02-01 23:59:39+00:00,,coronavirus venerd record 46 morti 2102 nuovi ...,Coronavirus venerd record con 46 morti e 2102 ...
3,1223757835214913536,2020-02-01 23:59:38+00:00,,coronavirus sappiamo resta ancora capire,Coronavirus tutto quello che sappiamo e ci che...
4,1223757834883674112,2020-02-01 23:59:38+00:00,,vabb pi morti femminicidio incidenti stradali ...,Si vabb ci sono pi morti per femminicidio o pe...
5,1223757834602647552,2020-02-01 23:59:38+00:00,,coronavirus gi 12mila casi mondo dieci paesi p...,Coronavirus gi 12mila casi nel mondo i dieci P...
6,1223757834065797120,2020-02-01 23:59:38+00:00,,coronavirus mascherina serve indossata male,Coronavirus la mascherina serve ma se indossat...
7,1223757833046515712,2020-02-01 23:59:38+00:00,,coronavirus dopo due casi accertati 32 persone...,Coronavirus dopo i due casi accertati 32 perso...
8,1223757821407264768,2020-02-01 23:59:35+00:00,#coronavirus,parente amico conoscente andando pi mangiare r...,Se avete un parente un amico o un conoscente c...
9,1223757748585816064,2020-02-01 23:59:18+00:00,#coronarvirusitalia #coronavirus,quando preoccupati corona virus no perch tanto...,Quando tutti sono preoccupati per il corona vi...


## Exporting

In [19]:
# Saving to CSV
tweets.to_csv('dataCleaned.csv')